In [0]:
# Imports
from pyspark.sql.functions import col, year, month, avg, sum, countDistinct, desc
from delta.tables import DeltaTable


### Step 1 — Bronze Layer: Raw Ingestion

**Tasks:**
- Read CSV and JSON data into DataFrames.
- Write them as Delta tables ( bronze_patients , bronze_hospitals ,
- bronze_appointments ).

In [0]:
# Read CSV and JSON files from DBFS or mounted storage
df_patients = spark.read.option("header", True).csv("/FileStore/tables/patients.csv")
df_hospitals = spark.read.option("multiline", True).json("/FileStore/tables/hospitals.json")
df_appointments_day1 = spark.read.option("header", True).csv("/FileStore/tables/appointments_day1.csv")

# Save to Delta Lake
df_patients.write.format("delta").mode("overwrite").saveAsTable("bronze_patients")
df_hospitals.write.format("delta").mode("overwrite").saveAsTable("bronze_hospitals")
df_appointments_day1.write.format("delta").mode("overwrite").saveAsTable("bronze_appointments")


### Step 2 — Silver Layer: Data Cleansing & Transformation

**Tasks:**
- Filter out Pending appointments.
- Join patients and hospitals to enrich appointment data.
- Add new calculated column: year = year(appointment_date) and month .
- Store output as silver_appointments .

In [0]:
# Load bronze tables
bronze_patients = spark.table("bronze_patients")
bronze_hospitals = spark.table("bronze_hospitals").withColumnRenamed("region", "hospital_region")
bronze_appointments = spark.table("bronze_appointments")

# Filter completed appointments
filtered_appointments = bronze_appointments.filter(col("status") == "Completed")

# Enrich with patient and hospital data
silver_appointments = filtered_appointments \
    .join(bronze_patients, "patient_id") \
    .join(bronze_hospitals, "hospital_id") \
    .withColumn("year", year("appointment_date")) \
    .withColumn("month", month("appointment_date"))

# Write to Silver table
silver_appointments.write.format("delta").mode("overwrite").saveAsTable("silver_appointments")


### Step 3 — Gold Layer: Analytical Aggregations
### Tasks:
- Total revenue per hospital.
- Total patients per region.
- Top 3 most expensive diagnosis categories.
- Store as gold_healthcare_summary

In [0]:
#1. Revenue Per Hospital
gold_revenue = silver_appointments.groupBy("hospital_id", "hospital_name") \
    .agg(sum("cost").alias("total_revenue"))

gold_revenue.write.format("delta").mode("overwrite").saveAsTable("gold_revenue_per_hospital")

#2. Patients Per Region
gold_patients = silver_appointments.select("patient_id", "region").distinct() \
    .groupBy("region").agg(countDistinct("patient_id").alias("total_patients"))

gold_patients.write.format("delta").mode("overwrite").saveAsTable("gold_patients_per_region")

#3. Top 3 Most Expensive Diagnoses
gold_diagnosis = silver_appointments.groupBy("diagnosis") \
    .agg(avg("cost").alias("avg_cost")) \
    .orderBy(desc("avg_cost")).limit(3)

gold_diagnosis.write.format("delta").mode("overwrite").saveAsTable("gold_top_diagnoses")

### Step 4 — Incremental Load Simulation
**Tasks:**
- Create appointments_day2.csv with new data.
- Use MERGE or Upsert to update the silver table.
- Show how incremental data changes the gold table.

In [0]:
data = """appointment_id,patient_id,hospital_id,appointment_date,diagnosis,cost,status
A1005,P001,H001,2024-02-10,Hypertension,600,Completed
A1006,P003,H003,2024-02-11,Heart Disease,1200,Completed
A1007,P002,H002,2024-02-15,Flu,200,Pending
A1008,P004,H004,2024-02-18,Allergy,350,Completed
"""

# Write the data to DBFS file path
dbutils.fs.put("/FileStore/tables/appointments_day2.csv", data, overwrite=True)


Wrote 279 bytes.


True

In [0]:

# STEP 1: Load new appointments data
new_appointments = spark.read.csv(
    "/FileStore/tables/appointments_day2.csv",
    header=True,
    inferSchema=True
)

# STEP 2: Filter out 'Pending' status
new_appointments_filtered = new_appointments.filter(
    col("status") != "Pending"
)

# STEP 3: Load bronze tables for enrichment
bronze_patients = spark.table("bronze_patients")
bronze_hospitals = spark.table("bronze_hospitals").withColumnRenamed("region", "hospital_region")

# STEP 4: Join & Enrich data with patients and hospitals
enriched_new_data = new_appointments_filtered \
    .join(bronze_patients, on="patient_id") \
    .join(bronze_hospitals, on="hospital_id") \
    .withColumn("year", year("appointment_date")) \
    .withColumn("month", month("appointment_date"))

# STEP 5: Load the silver Delta table
silver_table = DeltaTable.forName(spark, "silver_appointments")

# STEP 6: Define dictionary for merge column mapping
update_dict = {
    "appointment_id": "source.appointment_id",
    "patient_id": "source.patient_id",
    "hospital_id": "source.hospital_id",
    "appointment_date": "source.appointment_date",
    "diagnosis": "source.diagnosis",
    "cost": "source.cost",
    "status": "source.status",
    "name": "source.name",
    "age": "source.age",
    "gender": "source.gender",
    "region": "source.region",
    "hospital_name": "source.hospital_name",
    "hospital_region": "source.hospital_region",
    "year": "source.year",
    "month": "source.month"
}

# STEP 7: Perform UPSERT (merge)
silver_table.alias("target").merge(
    enriched_new_data.alias("source"),
    "target.appointment_id = source.appointment_id"
).whenMatchedUpdate(
    set=update_dict
).whenNotMatchedInsert(
    values=update_dict
).execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

### Step 5 — Delta Lake Features
**Tasks:**
- Use Time Travel to view the gold table before incremental load.
- Use Vacuum to clean up historical versions.
- Use Optimize + Z-Ordering on hospital_id .

In [0]:
#Time Travel
display(
    spark.sql(
        "SELECT * FROM gold_revenue_per_hospital VERSION AS OF 0"
    )
)

# Vacuum: clean up old versions (recommended minimum is 168 hours)
spark.sql(
    "VACUUM gold_revenue_per_hospital RETAIN 168 HOURS"
)

# Optimize + Z-ORDER
spark.sql(
    "OPTIMIZE gold_revenue_per_hospital ZORDER BY (hospital_name)"
)

hospital_id,hospital_name,total_revenue
H001,City Care,400.0
H004,CureWell,300.0
H002,LifePlus,250.0


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

**Analytical Questions to Solve**
1. Total revenue generated by each hospital.
2. Average cost per diagnosis category.
3. Number of patients served per region.
4. Trend of appointments month-over-month.
5. Top 5 most expensive treatments in the last 6 months.

In [0]:
#  Total revenue generated by each hospital
spark.sql("SELECT hospital_name, SUM(cost) AS total_revenue FROM silver_appointments GROUP BY hospital_name").show()

#  Average cost per diagnosis
spark.sql("SELECT diagnosis, AVG(cost) AS avg_cost FROM silver_appointments GROUP BY diagnosis").show()

#  Number of patients served per region
spark.sql("SELECT region, COUNT(DISTINCT patient_id) AS total_patients FROM silver_appointments GROUP BY region").show()

#  Trend of appointments month-over-month
spark.sql("SELECT year, month, COUNT(appointment_id) AS total_appointments FROM silver_appointments GROUP BY year, month ORDER BY year, month").show()

#  Top 5 most expensive treatments (last 6 months)
spark.sql("SELECT diagnosis, SUM(cost) AS total_cost FROM silver_appointments GROUP BY diagnosis ORDER BY total_cost DESC LIMIT 5").show()

     

+-------------+-------------+
|hospital_name|total_revenue|
+-------------+-------------+
|    City Care|       1000.0|
|     MediHope|       1200.0|
|     CureWell|        650.0|
|     LifePlus|        250.0|
+-------------+-------------+

+-------------+--------+
|    diagnosis|avg_cost|
+-------------+--------+
| Hypertension|   600.0|
|Heart Disease|  1200.0|
|      Allergy|   325.0|
|     Diabetes|   400.0|
|          Flu|   250.0|
+-------------+--------+

+------+--------------+
|region|total_patients|
+------+--------------+
| North|             1|
|  East|             1|
|  West|             1|
| South|             1|
+------+--------------+

+----+-----+------------------+
|year|month|total_appointments|
+----+-----+------------------+
|2024|    1|                 3|
|2024|    2|                 3|
+----+-----+------------------+

+-------------+----------+
|    diagnosis|total_cost|
+-------------+----------+
|Heart Disease|    1200.0|
|      Allergy|     650.0|
| Hypertensi